<center>
    <h1> Reddit Classificator </h1>
</center>





<br>
<br>
<center>
<img src="images/startgame.jpg" width=1000 height=500 /> </center>

# Abstract

In questo tutorial si vuole costruire un sistema di classificazione real-time che permetta di suddividere i post di reddit, sulla base del contenuto dei loro titoli, in categorie, per poi visualizzarne graficamente i risultati. 

# Data flow


<center>
<img src="images/dataFlow.png" width=1200 height=700 /> </center>
